In [2]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [10]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [11]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [6]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [7]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [8]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [9]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [30]:
# your code here
holder = 8
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
f'type=data&subject=writing&grade={holder}&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')

## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [31]:
## get the key
API_KEY = "1Gh9eKod5_-vX1otu_KEzcFphmJC58Kwb2qsJbcKPn76WUI7eW6biYfFe6R6Pmx_7NRaaQraF-Co_eLwcjvLl-DGQ4A0wHE8Z6WlFm9Bum6lEFvh9wiLufavouMtZXYx"

In [37]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [38]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 238,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.5,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,238,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.5,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,500,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
2,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,19,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.0,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552
3,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,False,https://www.yelp.com/biz/murphy-s-on-the-green...,173,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 43.702, 'longitude': -72.2894949}",[delivery],$$,"{'address1': '11 S Main St', 'address2': '', '...",+16036434075,(603) 643-4075,295.207523
4,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,374,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,244.006059


In [36]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,phone,display_phone,price
SyC7-D6pVpnm3tumaXMSeg,SyC7-D6pVpnm3tumaXMSeg,evettes-chicago-chicago-2,Evette's - Chicago,https://s3-media1.fl.yelpcdn.com/bphoto/D4zlA6...,https://www.yelp.com/biz/evettes-chicago-chica...,+17737998478,(773) 799-8478,NaN
Yc_VAHPXcl_h2CeEr_XfkQ,Yc_VAHPXcl_h2CeEr_XfkQ,bolivar-and-lincoln-venezuelan-cafe-chicago,Bolivar & Lincoln Venezuelan Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/Q__lWO...,https://www.yelp.com/biz/bolivar-and-lincoln-v...,+17733608203,(773) 360-8203,NaN
sJOPkTGLi53eB46ykvDeRg,sJOPkTGLi53eB46ykvDeRg,farm-bar-lakeview-chicago,Farm Bar Lakeview,https://s3-media1.fl.yelpcdn.com/bphoto/Vnonmr...,https://www.yelp.com/biz/farm-bar-lakeview-chi...,+17732812599,(773) 281-2599,$$
iyilEJb1NwUeZcd5JWXTKw,iyilEJb1NwUeZcd5JWXTKw,figo-wine-bar-chicago,Figo Wine Bar,https://s3-media4.fl.yelpcdn.com/bphoto/fQsMqb...,https://www.yelp.com/biz/figo-wine-bar-chicago...,+13128196111,(312) 819-6111,$$
-otD9myzwCgSYXdcx90jLQ,-otD9myzwCgSYXdcx90jLQ,dear-margaret-chicago,Dear Margaret,https://s3-media4.fl.yelpcdn.com/bphoto/qXWyjX...,https://www.yelp.com/biz/dear-margaret-chicago...,+17733608213,(773) 360-8213,$$$
IMZE58oaY-gRlv3qK9BCdw,IMZE58oaY-gRlv3qK9BCdw,tuk-tuk-thai-isan-street-food-chicago,Tuk Tuk Thai Isan Street Food,https://s3-media2.fl.yelpcdn.com/bphoto/W8L_dz...,https://www.yelp.com/biz/tuk-tuk-thai-isan-str...,+17736619739,(773) 661-9739,$$
T2Gcdw73pfSsVaK0hu9i6g,T2Gcdw73pfSsVaK0hu9i6g,bodega-sur-chicago,Bodega Sur,https://s3-media2.fl.yelpcdn.com/bphoto/uO5_7e...,https://www.yelp.com/biz/bodega-sur-chicago?ad...,+17739048656,(773) 904-8656,$$


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [46]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Chicago,IL,60657"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()

<Response [200]>

In [48]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'SyC7-D6pVpnm3tumaXMSeg',
 'alias': 'evettes-chicago-chicago-2',
 'name': "Evette's - Chicago",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/D4zlA6uznvGEhsWakmyJnQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/evettes-chicago-chicago-2?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 14,
 'categories': [{'alias': 'salad', 'title': 'Salad'},
  {'alias': 'tacos', 'title': 'Tacos'},
  {'alias': 'wraps', 'title': 'Wraps'}],
 'rating': 4.5,
 'coordinates': {'latitude': 41.933034, 'longitude': -87.65355},
 'transactions': [],
 'location': {'address1': '2807 N Sheffield Ave',
  'address2': None,
  'address3': '',
  'city': 'Chicago',
  'zip_code': '60657',
  'country': 'US',
  'state': 'IL',
  'display_address': ['2807 N Sheffield Ave', 'Chicago, IL 60657']},
 'phone': '+17737998478',
 'display_phone': '(773) 799-8478',
 'distance': 756.1726188270798}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,SyC7-D6pVpnm3tumaXMSeg,evettes-chicago-chicago-2,Evette's - Chicago,https://s3-media1.fl.yelpcdn.com/bphoto/D4zlA6...,False,https://www.yelp.com/biz/evettes-chicago-chica...,14,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,"{'latitude': 41.933034, 'longitude': -87.65355}",[],"{'address1': '2807 N Sheffield Ave', 'address2...",+17737998478,(773) 799-8478,756.172619,NaN
1,Yc_VAHPXcl_h2CeEr_XfkQ,bolivar-and-lincoln-venezuelan-cafe-chicago,Bolivar & Lincoln Venezuelan Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/Q__lWO...,False,https://www.yelp.com/biz/bolivar-and-lincoln-v...,211,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 41.943218, 'longitude': -87.65401}","[delivery, pickup]","{'address1': '3349 N Sheffield Ave', 'address2...",+17733608203,(773) 360-8203,377.272295,NaN
2,sJOPkTGLi53eB46ykvDeRg,farm-bar-lakeview-chicago,Farm Bar Lakeview,https://s3-media1.fl.yelpcdn.com/bphoto/Vnonmr...,False,https://www.yelp.com/biz/farm-bar-lakeview-chi...,459,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 41.93643, 'longitude': -87.66141}","[delivery, pickup]","{'address1': '1300 W Wellington Ave', 'address...",+17732812599,(773) 281-2599,731.699532,$$
3,iyilEJb1NwUeZcd5JWXTKw,figo-wine-bar-chicago,Figo Wine Bar,https://s3-media4.fl.yelpcdn.com/bphoto/fQsMqb...,False,https://www.yelp.com/biz/figo-wine-bar-chicago...,277,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 41.94015, 'longitude': -87.65386}",[delivery],"{'address1': '3207 N Sheffield Ave', 'address2...",+13128196111,(312) 819-6111,43.936284,$$
4,-otD9myzwCgSYXdcx90jLQ,dear-margaret-chicago,Dear Margaret,https://s3-media4.fl.yelpcdn.com/bphoto/qXWyjX...,False,https://www.yelp.com/biz/dear-margaret-chicago...,117,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 41.93589, 'longitude': -87.66294}",[delivery],"{'address1': '2965 N Lincoln Ave', 'address2':...",+17733608213,(773) 360-8213,859.967136,$$$


In [50]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)

,id,alias,name,image_url,url,phone,display_phone,price
SyC7-D6pVpnm3tumaXMSeg,SyC7-D6pVpnm3tumaXMSeg,evettes-chicago-chicago-2,Evette's - Chicago,https://s3-media1.fl.yelpcdn.com/bphoto/D4zlA6...,https://www.yelp.com/biz/evettes-chicago-chica...,+17737998478,(773) 799-8478,NaN
Yc_VAHPXcl_h2CeEr_XfkQ,Yc_VAHPXcl_h2CeEr_XfkQ,bolivar-and-lincoln-venezuelan-cafe-chicago,Bolivar & Lincoln Venezuelan Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/Q__lWO...,https://www.yelp.com/biz/bolivar-and-lincoln-v...,+17733608203,(773) 360-8203,NaN
sJOPkTGLi53eB46ykvDeRg,sJOPkTGLi53eB46ykvDeRg,farm-bar-lakeview-chicago,Farm Bar Lakeview,https://s3-media1.fl.yelpcdn.com/bphoto/Vnonmr...,https://www.yelp.com/biz/farm-bar-lakeview-chi...,+17732812599,(773) 281-2599,$$
iyilEJb1NwUeZcd5JWXTKw,iyilEJb1NwUeZcd5JWXTKw,figo-wine-bar-chicago,Figo Wine Bar,https://s3-media4.fl.yelpcdn.com/bphoto/fQsMqb...,https://www.yelp.com/biz/figo-wine-bar-chicago...,+13128196111,(312) 819-6111,$$
-otD9myzwCgSYXdcx90jLQ,-otD9myzwCgSYXdcx90jLQ,dear-margaret-chicago,Dear Margaret,https://s3-media4.fl.yelpcdn.com/bphoto/qXWyjX...,https://www.yelp.com/biz/dear-margaret-chicago...,+17733608213,(773) 360-8213,$$$
IMZE58oaY-gRlv3qK9BCdw,IMZE58oaY-gRlv3qK9BCdw,tuk-tuk-thai-isan-street-food-chicago,Tuk Tuk Thai Isan Street Food,https://s3-media2.fl.yelpcdn.com/bphoto/W8L_dz...,https://www.yelp.com/biz/tuk-tuk-thai-isan-str...,+17736619739,(773) 661-9739,$$
T2Gcdw73pfSsVaK0hu9i6g,T2Gcdw73pfSsVaK0hu9i6g,bodega-sur-chicago,Bodega Sur,https://s3-media2.fl.yelpcdn.com/bphoto/uO5_7e...,https://www.yelp.com/biz/bodega-sur-chicago?ad...,+17739048656,(773) 904-8656,$$


In [ ]:
yelp_stronly_df['reviews'] = 

In [ ]:
# your code here